In [ ]:
using Pkg
Pkg.activate("/Users/bbrener1/haxx/RNAForecasterPaperCode/")

In [ ]:
include("../trainRNAForecasterV2.jl")

In [ ]:
include("../recursivePredictionsV2.jl")

In [ ]:
using DelimitedFiles
spliced = readdlm("pancSplicedCountsHVGs.csv", ',')
unspliced = readdlm("pancUnsplicedCountsHVGs.csv", ',')

zeroPropSplicedGenes = Array{Float64}(undef, size(spliced)[1])
for i=1:size(spliced)[1]
    zeroPropSplicedGenes[i] = length(findall(x->x ==0, spliced[i,:]))/size(spliced)[2]
end
zeroPropUnsplicedGenes = Array{Float64}(undef, size(unspliced)[1])
for i=1:size(unspliced)[1]
    zeroPropUnsplicedGenes[i] = length(findall(x->x ==0, unspliced[i,:]))/size(unspliced)[2]
end

splicedSub = spliced[intersect(findall(x->x < 0.98, zeroPropSplicedGenes), findall(x->x < 0.98, zeroPropUnsplicedGenes)),:]
unsplicedSub = unspliced[intersect(findall(x->x < 0.98, zeroPropSplicedGenes), findall(x->x < 0.98, zeroPropUnsplicedGenes)),:]

geneNames = readdlm("pancHVGNames.csv", ',')
geneNames = geneNames[intersect(findall(x->x < 0.98, zeroPropSplicedGenes), findall(x->x < 0.98, zeroPropUnsplicedGenes))]
geneNames = string.(geneNames)

writedlm("ZPSGenes.tsv",geneNames)

In [ ]:
# Let's load a trained model
using Flux: loadmodel!
using JLD2
function loadForecaster(fileName::String, inputNodes::Int, hiddenLayerNodes::Int)
    #recreate neural network structure
    nn = Chain(Dense(inputNodes, hiddenLayerNodes, relu),
               Dense(hiddenLayerNodes, inputNodes))
    model = NeuralODE(nn, (0.0f0, 1.0f0), Tsit5(),
                       save_everystep = false,
                       reltol = 1e-3, abstol = 1e-3,
                       save_start = false)
    #load parameters into the model
    model = loadmodel!(model, load_object(fileName))
    return model
end



In [ ]:
trained = loadForecaster("pancNeuralODEResult.jld2", 1352, 6000)

In [ ]:
target = Float32.(splicedSub)
size(target)

In [ ]:
include("../recursivePredictionsV2.jl")

In [ ]:
futures = predictCellFutures(trained,target,3,damping=1f-2)

In [ ]:
writedlm("./trajectories_t1.tsv",futures[:,:,1])
writedlm("./trajectories_t2.tsv",futures[:,:,2])
writedlm("./trajectories_t3.tsv",futures[:,:,3])

In [ ]:
ob = Flux.state(trained)

In [ ]:
ob.p